In [45]:
## DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [1]:
import torch    
import torch.nn as nn
import torch.nn.functional as F
import torch.fft
from layers.Embed import DataEmbedding
from layers.Conv_Blocks import Inception_Block_V1   
            #convolution block used for convoluting the 2D time data, changeable

/Users/hyeongyukang/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from utils import data_split

### train + test(4년간 3월 평균)

In [16]:
unique_code = ['BC_C_J', 'TG_B_J', 'CR_B_J', 'RD_E_S', 'BC_A_J', 'CB_F_J', 'RD_D_J', 'TG_A_S', 'BC_E_S', 'CR_D_J', 'BC_A_S', 'BC_B_S', 'TG_E_J', 
               'CR_E_S', 'RD_F_J', 'BC_E_J', 'TG_A_J', 'CR_C_J', 'CR_D_S', 'TG_C_J', 'CB_A_S', 'TG_D_J', 'CR_E_J', 'RD_C_S', 'BC_C_S', 'CB_E_J', 
               'RD_E_J', 'BC_D_J', 'CR_A_J', 'TG_E_S', 'TG_C_S', 'TG_D_S', 'RD_A_S', 'RD_A_J', 'RD_D_S', 'TG_B_S', 'CB_D_J', 'CB_A_J', 'BC_B_J']

data = pd.read_csv('~/Developer/private/Dacon/jeju/data/train.csv')
data_list = data_split(data)

dataset = {}
march_data = {}
for code in unique_code:
    march_data[code] = {}
    march_data[code]['2019'] = data_list[f'data_{code}'][(data_list[f'data_{code}']['timestamp'] >= '2019-03-04') & (data_list[f'data_{code}']['timestamp'] <= '2019-03-31')].reset_index(drop = True)
    march_data[code]['2020'] = data_list[f'data_{code}'][(data_list[f'data_{code}']['timestamp'] >= '2020-03-04') & (data_list[f'data_{code}']['timestamp'] <= '2020-03-31')].reset_index(drop = True)
    march_data[code]['2021'] = data_list[f'data_{code}'][(data_list[f'data_{code}']['timestamp'] >= '2021-03-04') & (data_list[f'data_{code}']['timestamp'] <= '2021-03-31')].reset_index(drop = True)
    march_data[code]['2022'] = data_list[f'data_{code}'][(data_list[f'data_{code}']['timestamp'] >= '2022-03-04') & (data_list[f'data_{code}']['timestamp'] <= '2022-03-31')].reset_index(drop = True)
    

    avg_supply = []
    for i in range(28):
        supply_19 = march_data[code]['2019']['supply(kg)'][i]
        supply_20 = march_data[code]['2020']['supply(kg)'][i] 
        supply_21 = march_data[code]['2021']['supply(kg)'][i] 
        supply_22 =march_data[code]['2022']['supply(kg)'][i] 

        supplies = [supply_19, supply_20, supply_21, supply_22]
        filtered_supplies = [supply for supply in supplies if supply != 0]

        # 평균 계산
        if filtered_supplies:
            average_supply = sum(filtered_supplies) / len(filtered_supplies)
        else: 
            average_supply = 0
        
        avg_supply.append(average_supply)

    zero_sunday = [1, 8, 15, 22]
    for idx in zero_sunday:
        avg_supply[idx] = 0


    avg_price = []
    for i in range(28):
        price_19 = march_data[code]['2019']['price(원/kg)'][i]
        price_20 = march_data[code]['2020']['price(원/kg)'][i] 
        price_21 = march_data[code]['2021']['price(원/kg)'][i] 
        price_22 =march_data[code]['2022']['price(원/kg)'][i] 

        prices = [price_19, price_20, price_21, price_22]
        filtered_prices = [price for price in prices if price != 0]

        # 평균 계산
        if filtered_prices:
            average_price = sum(filtered_prices) / len(filtered_prices)
        else: 
            average_price = 0
        
        avg_price.append(average_price)
        

    zero_sunday = [1, 8, 15, 22]
    for idx in zero_sunday:
        avg_price[idx] = 0


    march_data[code]['avg_supply'] = avg_supply
    march_data[code]['avg_price'] = avg_price

    # 2023년 3월 test용 데이터셋 생성
    new_df = march_data[code]['2019'].copy()
    # 'ID'의 연도를 2023으로 변경
    new_df['ID'] = new_df['ID'].apply(lambda x: f"{x[:5]}2023{x[9:]}")
    # 'timestamp'의 연도를 2023으로 변경
    new_df['timestamp'] = new_df['timestamp'].apply(lambda x: f"2023-{x[5:]}")
    # supply와 price의 평균값으로 변경
    new_df['supply(kg)'] = avg_supply
    new_df['price(원/kg)'] = avg_price

    march_data[code]['2023'] = new_df
    

    # 원래 데이터와 합침
    merged_df = pd.concat([data_list[f'data_{code}'], march_data[code]['2023']], axis=0, ignore_index=True)
    dataset[code] = merged_df

In [18]:
dataset['TG_A_S']

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_S_20190101,2019-01-01,TG,A,S,0.000000,0.000000
1,TG_A_S_20190102,2019-01-02,TG,A,S,0.000000,0.000000
2,TG_A_S_20190103,2019-01-03,TG,A,S,190591.000000,2526.000000
3,TG_A_S_20190104,2019-01-04,TG,A,S,137729.000000,2134.000000
4,TG_A_S_20190105,2019-01-05,TG,A,S,134039.000000,2075.000000
...,...,...,...,...,...,...,...
1546,TG_A_2023190327,2023-03-27,TG,A,S,9511.000000,5015.666667
1547,TG_A_2023190328,2023-03-28,TG,A,S,16388.000000,5254.333333
1548,TG_A_2023190329,2023-03-29,TG,A,S,16599.666667,4729.000000
1549,TG_A_2023190330,2023-03-30,TG,A,S,12938.125000,4760.250000


### DataLoader

In [20]:
seq_len = 64
label_len = 28
pred_len = 28



type_map = {'train': 0, 'val': 1, 'test': 2}
flag = 'train'
set_type = type_map[flag]
features = 'S' # single or multi
target = 'price(원/kg)' 
scale = True
timeenc = 0 # time_feature가 존재하는지 여부 : 없으면 임의로 생성해주지만 우리 데이터에는 존재함
freq = 'h'

In [37]:
scaler = StandardScaler()

#get raw data from path
df_raw = dataset['TG_A_J']

# TVT
border1s = [0, 1400 - seq_len, 1523 - seq_len] # x for train valid test
border2s = [1400, 1523, 1551] # y for train valid test
border1 = border1s[set_type]
border2 = border2s[set_type]

# redefine dataframe with target col
df_data = df_raw[[target]] 

# train data scaling
if scale:
    train_data = df_data[border1s[0]:border2s[0]]
    #train_data.values: turn pandas DataFrame into 2D numpy
    scaler.fit(train_data.values)  
    data = scaler.transform(df_data.values)
else:
    data = df_data.values 


df_stamp = df_raw[['timestamp']][border1:border2]
    
# 날짜를 년/월/일/요일 형태로 자르고 리스트로 변환

# [[2019    1    1    1]
#  [2019    1    2    2]
#  [2019    1    3    3]
#  ...

df_stamp['timestamp'] = pd.to_datetime(df_stamp.timestamp) 

if timeenc == 0:  #time feature encoding is fixed or learned
    df_stamp['year'] = df_stamp.timestamp.apply(lambda row: row.year, 1)
    df_stamp['month'] = df_stamp.timestamp.apply(lambda row: row.month, 1)
    df_stamp['day'] = df_stamp.timestamp.apply(lambda row: row.day, 1)
    df_stamp['weekday'] = df_stamp.timestamp.apply(lambda row: row.weekday(), 1)
    data_stamp = df_stamp.drop(['timestamp'], axis = 1).values



# data_x and data_y are same copy of a certain part of data
data_x = data[border1:border2]
data_y = data[border1:border2]
data_stamp = data_stamp



/var/folders/6r/w8k58h7j3799_jkwn2c_nk7c0000gn/T/ipykernel_49718/3237434334.py:37: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['year'] = df_stamp.timestamp.apply(lambda row: row.year, 1)
/var/folders/6r/w8k58h7j3799_jkwn2c_nk7c0000gn/T/ipykernel_49718/3237434334.py:38: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['month'] = df_stamp.timestamp.apply(lambda row: row.month, 1)
/var/folders/6r/w8k58h7j3799_jkwn2c_nk7c0000gn/T/ipykernel_49718/3237434334.py:39: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['day'] = df_stamp.timestamp.apply(lambda row: row.day, 1)
/var

In [39]:
def A(index):
    #given an index, calculate the positions after this index to truncate the dataset
    s_begin = index
    s_end = s_begin + seq_len
    r_begin = s_end - label_len
    r_end = r_begin + label_len + pred_len

    #input and output sequence
    seq_x = data_x[s_begin:s_end]
    seq_y = data_y[r_begin:r_end]

    #time mark
    seq_x_mark = data_stamp[s_begin:s_end]
    seq_y_mark = data_stamp[r_begin:r_end]

    return seq_x, seq_y, seq_x_mark, seq_y_mark


In [41]:
seq_x, seq_y, seq_x_mark, seq_y_mark = A(0)